In [ ]:
# import libraries

import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
#load dataset
df_train = pd.read_csv(train_file_path,sep="\t", header=None, names = ['label','text'])
df_test = pd.read_csv(test_file_path,sep="\t", header=None, names = ['label','text'])


In [ ]:
df_train.head()

In [ ]:
df_test.tail()

In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
#transformdata Categorical to Numeric
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
df_train['label'] = labelencoder.fit_transform(df_train['label'])
df_test['label'] = labelencoder.fit_transform(df_test['label'])


df_train.head()

In [ ]:
labels_train = df_train['label']
labels_test = df_test['label']

In [ ]:
labels_train.shape

In [ ]:
#visualization for each label
import seaborn as sns
plt.figure(figsize=(8,4))
sns.countplot(x='label', hue='label',data=df_train)

In [ ]:
#preprocessing word
import nltk
import requests
import string
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
#clean stopwords and text
stopList = set(stopwords.words('english'))

In [ ]:
lemmat = WordNetLemmatizer()

def cleaning(data):
  data = re.sub('[%s]' % re.escape(string.punctuation), ' ',data)
  data = str(data.lower()).strip()
  tokens=[]
  for token in data.split():
    if token not in stopList:
      token = lemmat.lemmatize(token)
      tokens.append(token)
  return " ".join(tokens)

In [ ]:
df_train['text'] = df_train.text.apply(lambda x: cleaning(x))

In [ ]:
df_train.head()

In [ ]:
df_test['text'] = df_test.text.apply(lambda x: cleaning(x))

In [ ]:
#vectorize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [ ]:
maxLen= 500
max_words = 1000

In [ ]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df_train.text)
X_train_seq = tokenizer.texts_to_sequences(df_train.text)
X_test_seq = tokenizer.texts_to_sequences(df_test.text)

In [ ]:
print(X_train_seq[:5])
print(df_train.iloc[0,:])



In [ ]:
X_train_pad = pad_sequences(X_train_seq, maxlen=maxLen, padding='pre')
X_test_pad = pad_sequences(X_test_seq, maxlen=maxLen, padding='pre')
X_train_pad.shape

In [ ]:
X_train_pad[:5]

In [ ]:
model = keras.Sequential([
    keras.layers.Embedding(max_words, 50, input_length=maxLen),
    keras.layers.LSTM(64),
    keras.layers.Dense(1, activation='sigmoid')
])


model.compile(
    loss='binary_crossentropy',
    optimizer='RMSprop',
    metrics=['accuracy']
)
model.summary()

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

In [ ]:
history = model.fit(X_train_pad, labels_train,
              batch_size=128, epochs= 10,
              validation_split=0.2,
             )

In [ ]:
plt.plot(history.history['acc'], label='accuracy')
plt.plot(history.history['val_acc'], label='val_acc')
plt.legend()

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
#evaluation
y_pred=model.predict(X_test_pad)
y_pred=(y_pred>0.5)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
score=accuracy_score(labels_test,y_pred)
print("Test Score:{:.2f}%".format(score*100))

In [ ]:
def pre_text(X):
  x = X.apply(lambda x: cleaning(x))
  x = tokenizer.texts_to_sequences(x)
  return pad_sequences(x, maxlen=maxLen)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  p = model.predict(pre_text(pd.Series([pred_text])))[0]

  return (p[0], ("ham" if p<0.5 else "spam"))

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
